In [1]:
# RUN THIS CODE BLOCK BY PRESSING THE PLAY BUTTON THAT SAYS RUN AND GO TO THE NEXT CODE BLOCK
# IF YOU WANT TO CHANGE/REFINE THE DEPTH OF THE SEARCH CHANGE THE TWO VARIABLES BELOW NAMED pageRange and pageSize

import requests
import pandas as pd
from bs4 import BeautifulSoup


pageRange = 5
pageSize = "10"

def singleLinkInfoListBuilder(URLextender):

    URL = "https://pubmed.ncbi.nlm.nih.gov/" + str(URLextender) + "/"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    tempInfoList = []
    finalInfoList = []
    
    
    paperTitle = paperTitleFinder(URL)
    paperID = IDGetter(URL)
    nameList = nameGetter(URL)

    results = soup.find(id = "full-view-expanded-authors")
    
    if len(soup.find_all(id = "full-view-expanded-authors")) == 0:
        finalInfoList.append("")
        return finalInfoList
    
    if (len(results.find_all("ul")) == 0):
        finalInfoList.append("")
        return finalInfoList
    
    if nameList == []:
        finalInfoList.append("")
        return finalInfoList
    
    results = results.ul
    results = results.find_all("li")

    tempList = [None] * len(results)
    

    for i in range(len(results)):
        tempList[i] = str(results[i])
        affiliationNumber = tempList[i][37]
        for j in nameList:
            if affiliationNumber in j[1]:
                person = j[0]
        if tempList[i][-6] == ".":
            tempList[i] = tempList[i][65:-6]
        else:
            tempList[i] = tempList[i][65:-5]
        tempList[i] = tempList[i].split(",")
        if "@" in tempList[i][-1]:
            
            email = finalEmailCleaner(tempList[i][-1])
            
            try:
                
                tempInfoList.append([person, tempList[i][0] + ", " + tempList[i][1], email])  
                
            except IndexError:
                
                print(paperTitle + "FAILED")
                print(tempList[i])
    
    if len(tempInfoList) == 0:
        finalInfoList.append("")
    
    else:
        finalInfoList.append(paperTitle)
        finalInfoList.append(paperID)
        finalInfoAppender = []
        for i in range(len(tempInfoList)):
            if tempInfoList[i] not in finalInfoAppender:
                finalInfoAppender.append(tempInfoList[i])
        finalInfoList.append(finalInfoAppender)    
    
    print(finalInfoList)
    return finalInfoList


def searchPageURLFinder(searchPageURL):
    
    resultsList = []
    for i in range(1, pageRange):
        
        searchPageURL += "&page=" + str(i)
        
        page = requests.get(searchPageURL)
        soup = BeautifulSoup(page.content, 'html.parser')
    
        results = soup.find_all(string = isADataChunkID)
    
        for i in results:
            resultsList.append(i)
    
    infoList = []
    
    for i in resultsList:
        infoList.append(singleLinkInfoListBuilder(i))
        
    return infoList
    
def isADataChunkID(s):
    
    while True:
        try:
            tempExt = int(s)
            return (len(str(tempExt)) == 8 or len(str(tempExt)) == 7 or len(str(tempExt)) == 6)
        except ValueError:
            return False
        
def getUserInput():
    
    search = str(input("Please enter your desired search term: "))
    listified = search.split(" ")
    result = ""
    for i in range(len(listified)):
        if i < len(listified) - 1:
            result += listified[i] + "+"
        else:
            result += listified[i]
    
    return "https://pubmed.ncbi.nlm.nih.gov/?term=" + result + "&size=" + pageSize + ""
                                 
def finalEmailCleaner(item):
                                 
    itemList = item.split(" ")
    return itemList[-1]
                                 
def paperTitleFinder(URL):
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find(class_ = "heading-title")
    results = str(results)
    results = results.split("\n")
    if len(results) == 1:
        return results[0]
    return results[7][2:]
              
    
def nameGetter(URL):
    
    affiliationNumbers = []
    nameList = []
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    results = soup.find(class_ = "inline-authors")
    try:
        results = results.find_all(class_ = "authors-list-item")
    except AttributeError:
        return nameList
    
    for i in range(len(results)):
        
        affiliationNumbers.append(results[i].find_all(class_ = "affiliation-link"))
        
    for i in range(len(affiliationNumbers)):
        for j in range(len(affiliationNumbers[i])):
            temp = str(affiliationNumbers[i][j])
            temp = temp[47]
            affiliationNumbers[i][j] = temp
    

    for i in range(len(results)):
    
        curr = -5
        end = -4
        bigName = results[i].a
        bigName = str(bigName)
        
        
        while bigName[curr] != ">":
            curr -= 1
        
        name = bigName[curr + 1:end]
        nameList.append([name, affiliationNumbers[i]])
    
    return nameList

def IDGetter(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    results = soup.find(class_ = "current-id")

    results = str(results)

    IDFinder = False
    IDnum = ""

    for char in results:
    
        if char == "<":
            IDFinder = False
        if IDFinder:
            IDnum += char
        if char == ">":
            IDFinder = True
        
    return IDnum

def finalInfoListCleaner(finalList):
    
    stillCleaning = True
    while stillCleaning:
        try:
            finalList.remove([''])
        except ValueError:
            stillCleaning = False
            
    return finalList

In [2]:
# THIS IS WHERE THE MOST ACTION HAPPENS. MAKE SURE YOU RUN THE CODE CELL ABOVE THIS ONE BEFORE RUNNING THIS ONE.
# RUNNING THIS ONE WILL FIRST PROMPT YOU FOR A SEARCH TERM. ENTER IT IN AND WATCH.

searchTopic = getUserInput()

emailList = searchPageURLFinder(searchTopic)

emailList = finalInfoListCleaner(emailList)

print(emailList)

Please enter your desired search term: Amyotrophic lateral sclerosis
['']
['Amyotrophic lateral sclerosis', '21296405', [['Matthew C Kiernan', 'Neuroscience Research Australia and Prince of Wales Clinical School,  University of New South Wales', 'm.kiernan@unsw.edu.au']]]
['']
['Amyotrophic lateral sclerosis: disease state overview', '30207670', [['Darrell Hulisz', 'Associate Professor,  Department of Family Medicine', 'dth4@case.edu']]]
['']
['Amyotrophic Lateral Sclerosis: An Update for 2018', '30401437', [['Björn Oskarsson', 'Department of Neurology,  Mayo Clinic', 'oskarsson.bjorn@mayo.edu']]]
['Neuropathology of Amyotrophic Lateral Sclerosis and Its Variants', '26515626', [['John Ravits', 'Department of Neurosciences,  ALS Translational Research', 'jravits@ucsd.edu']]]
['']
['']
['Care management in amyotrophic lateral sclerosis', '28461024', [['M-H Soriani', 'Centre de référence maladies neuromusculaire/SLA,  university hospital of Nice', 'soriani.mh@chu-nice.fr']]]
['Respiratory

In [6]:
# DON'T WORRY ABOUT HOW THIS WORKS OR WHAT IT DOES, IT JUST HELPS FOR FORMATTING THE OUTPUT.
# RUN THIS CELL AND CONTINUE

def nameSplitter(name):
    
    name = name.split(" ")
    return [name[0], name[-1]]

def exportListBuilder(emailList):
    
    exportList = []
    
    for i in range(len(emailList)): 
        
        for j in range(len(emailList[i][2])):
            
            nameList = nameSplitter(emailList[i][2][j][0])
    
            adder = [nameList[0], nameList[1], emailList[i][2][j][1], emailList[i][2][j][2], "", "", "", "", "", "KOL", "", emailList[i][0]]
            
            exportList.append(adder)
                               
    return exportList

formattedList = exportListBuilder(emailList)

In [7]:
# RUN THIS CODE BLOCK
# WE HAVE NOW CREATED A DATA STRUCTURE THAT WILL HELP US OUTPUT THE DATA

colNames = ["First Name", "Last Name", "Affiliation", "Email", "blank", "blank", "blank", "blank", "blank", "Type Of Consultant", "blank", "Specialty"]

exporter = pd.DataFrame(formattedList, columns = colNames)

print(exporter)


     First Name  Last Name                                        Affiliation  \
0       Matthew    Kiernan  Neuroscience Research Australia and Prince of ...   
1       Darrell     Hulisz  Associate Professor,  Department of Family Med...   
2         Björn  Oskarsson              Department of Neurology,  Mayo Clinic   
3          John     Ravits  Department of Neurosciences,  ALS Translationa...   
4           M-H    Soriani  Centre de référence maladies neuromusculaire/S...   
5        Philip       Choi  Division of Pulmonary and Critical Care,  Depa...   
6          Ruth       Chia  Laboratory of Neurogenetics,  National Institu...   
7          Nilo       Riva  Neuropathology Unit,  INSPE and Division of Ne...   
8       Michele      Lewis  Department of Physical Medicine and Rehabilita...   
9         Mazen  Dimachkie  Department of Neurology,  University of Kansas...   
10  Jean-Pierre   Bouchard  Institut psycho-judiciaire et de psychopatholo...   
11     Katerina     Kadena  

In [8]:
# CHANGE THE FILE PATH TO YOUR DESIRED ROUTE AND CHANAGE THE FILE NAME TO YOUR DESIRED NAME
# IMPORTANT!!!! IN ORDER FOR THIS TO OUTPUT SUCCESSFULLY, YOUR OUTPUT FOLDER MUST BE SAVED WITHIN THE SAME FOLDER
# THAT THIS CODE IS SAVED IN!!!!
# THEN RUN THIS CODE BLOCK

fileName = 'NewDemoSheet1'

exporter.to_excel(r'/Users/maxbronfeld/Desktop/PM/Scraper Work/Web Scraper/Output/' + fileName + '.xlsx', index = False, header = True)

['hello']
